In [244]:
#######                       #######
####### Presenting: Spellr.AI #######
#######                       #######

####### section 00
####### loading libraries


import itertools
import re
import math
from collections import Counter
from pprint import pprint


In [245]:

####### section 01
####### loading databases


dataset_lst = []
dataset_lst.append("yuge_data.txt")
dataset_lst.append("internet-en.num.txt")
dataset_lst.append("en.txt")
dataset_lst.append("words.txt")
dataset_lst.append("words_alpha.txt")
dataset_lst.append("english3.txt")
dataset_lst.append("english2.txt")
dataset_lst.append("usa.txt")

# these dont work (because they contain non utf-8 chars)
# dataset_lst.append("ukenglish.txt")
# dataset_lst.append("engmix.txt")
# dataset_lst.append("usa2.txt")

yuge_data = ""
cnt = 1
for d in dataset_lst:
    data_import = open("../datasets/"+d).read()
    yuge_data += data_import
    print(">>> importing  dataset #" + str(cnt))
    print(data_import[:64])
    print()

    cnt += 1


>>> importing  dataset #1
The Project Gutenberg EBook of The Adventures of Sherlock Holmes

>>> importing  dataset #2
The frequency distribution for attribute 'lemma' in corpus 'inte

>>> importing  dataset #3
you 6281002
i 5685306
the 4768490
to 3453407
a 3048287
it 287996

>>> importing  dataset #4
2
1080
&c
10-point
10th
11-point
12-point
16-point
18-point
1st


>>> importing  dataset #5
a
aa
aaa
aah
aahed
aahing
aahs
aal
aalii
aaliis
aals
aam
aani
aa

>>> importing  dataset #6
a
aa
aaa
aachen
aardvark
aardvarks
aardwolf
aardwolves
aarhus
aa

>>> importing  dataset #7
a
aa
aaa
aachen
aardvark
aardvarks
aaron
aback
abacus
abacuses
a

>>> importing  dataset #8
a
aa
aaa
aachen
aardvark
aardvarks
aaron
aback
abacus
abacuses
a



In [246]:

####### section 02
####### processing databases


def extract_words(src):
    res = src

    # distinguish between upper/lower case words
    res = re.findall(r'\b\w+\b', src)

    # filter out numbers
    res = list(itertools.filterfalse(lambda x: x.isnumeric(), res))

    # filter out words of length 1
    res = list(itertools.filterfalse(lambda x: len(x) == 1, res))

    return res

print("\n[i] the 1st 10 words in all the databases")
pprint( extract_words(yuge_data)[:10] )
WORD_IDX = Counter(extract_words(yuge_data))

print("\n[i] retrieve the most common words")
pprint( WORD_IDX.most_common(19) )

# get the sum of all words
ALL_WORD_CNT = sum(WORD_IDX.values())

# get the number of occurrences 
WORD_IDX.values()

# probability function
# used to help answer the question: how likely is a given word?
def ProbabilityForWord(word):
    return WORD_IDX[word]/ALL_WORD_CNT



[i] the 1st 10 words in all the databases
['The',
 'Project',
 'Gutenberg',
 'EBook',
 'of',
 'The',
 'Adventures',
 'of',
 'Sherlock',
 'Holmes']

[i] retrieve the most common words
[('the', 72390),
 ('of', 39844),
 ('and', 37067),
 ('to', 28370),
 ('in', 20227),
 ('that', 11994),
 ('was', 11372),
 ('he', 10062),
 ('is', 9648),
 ('his', 9628),
 ('with', 9501),
 ('it', 8572),
 ('as', 7487),
 ('had', 7337),
 ('The', 7245),
 ('for', 6671),
 ('by', 6651),
 ('not', 6485),
 ('at', 6360)]


In [247]:

####### section 03
####### building a candidate model
        # (this boils down to generating possible 
        #  words with simple string manipulation)


# === helper functions
# flatten a list of two tuples
def lst_flatten(lst):
    ret = []
    for a,b in lst:
        ret.append(a)
        ret.append(b)
    return ret

# can a given word be found in the "dictionary"?
def only_words_in_dict(words):
    assert( type(words) == set )
    return set(w for w in words if WORD_IDX[w] > 0)



def candidates_1(word):
    # "".join([chr(c) for c in range(65, 91)])
    alphab = 'abcdefghijklmnopqrstuvwxyz'+'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

    # edits formed by inserting a space
    s = [(word[:i], word[i:]) for i in range(len(word) + 1)]

    # edits formed by deleting a character
    d = [L + R[1:] for L, R in s if R]

    # edits formed by swapping the i'th and (i+1)'th characters
    # where i < len(word) - 1
    t = [L + R[1] + R[0] + R[2:] for L, R in s if len(R)>1] 
    # ?? consider adding transposes of a longer length 
    # (i.e., three or four letter transposes)

    # edits formed by substituting a random character
    # (similar to: "edits formed by inserting a space" as shown above)
    r = []
    for L, R in s:
        if R:
            for c in alphab:
                r.append( L + c + R[1:] )

    # edits formed by inserting a random char
    i = [L + c + R for L, R in s for c in alphab]

    # dont exclude split words
    # consider them as candidates
    s = lst_flatten(s)

    # return all: splits, deletes, edits, replacements, and insertions
    return set(s + d + t + r + i)

# This code kinda works, but note that when n >= 3, the function "blows up".
# Finding all edits of distance >= 3 is computationally hard.
def candidates_n(word, n):
    if (n == 1):
        return candidates_1(word)
    else: 
        res = set()
        for a in candidates_n(word, n-1):
            
            # Note: the for loop below is not part of the
            # recursion. it's used for flattening the set.
            for z in candidates_1(a):
                res.add(z)
        return res

def candidates(word):
    res = set()
    for e1 in only_words_in_dict(candidates_n(word, 1)):
        res.add(e1)
    for e2 in only_words_in_dict(candidates_n(word, 2)):
        res.add(e2)

    # res.add(word)

    # sort according to the following:
        # favour longer words over shorter ones
        # favour more frequent words over less frequent words
    res = sorted(res, key=lambda elmt:(len(elmt), WORD_IDX[elmt]), reverse=True)

    return res

def best_guess(word):
    return candidates(word)[0]



In [248]:
####### section 04
####### unit testing


print("[i] ProbabilityForWord(\"the\"):", ProbabilityForWord('the'))
print("[i] ProbabilityForWord(\"pineapple\"):", ProbabilityForWord("pineapple"))
print("[i] pineapple occurs", WORD_IDX["pineapple"], "times in all datasets")
print("[i] all words:", ALL_WORD_CNT)

# assertions
assert(candidates_1("pineapple") == candidates_n("pineapple", 1))

for upper_case_letter in range(65, 91): # A = 65, # Z = 90
    assert( ProbabilityForWord(upper_case_letter) < 1 )
    assert( ProbabilityForWord(upper_case_letter) >= 0 )
    assert( WORD_IDX[upper_case_letter] < ALL_WORD_CNT )

for lower_case_letter in range(97, 123): # a = 97, # z = 122
    assert( ProbabilityForWord(lower_case_letter) < 1 )
    assert( ProbabilityForWord(lower_case_letter) >= 0 )
    assert( WORD_IDX[lower_case_letter] < ALL_WORD_CNT )

# ensure that the most common words 
# have reasonable probabilities
common_words = [elmt[0] for elmt in WORD_IDX.most_common(100)]
for c in common_words:
    assert( ProbabilityForWord(c) < 1)

# for each of these words, generate the best seven guesses

guess = []
guess.append("toothhick")
guess.append("princeron")
guess.append("jupytir")
guess.append("mashuo")
guess.append("pineappee")
guess.append("that")
guess.append("peele")

for g in guess:
    print( "\n[word: " + g + "] best guess: " + best_guess(g) )
    print( "probability: " 
          + str( ProbabilityForWord(best_guess(g)) ) 
          + "" )

    print( "[word: " + g + "] 5 other possible guesses " + g )
    for x in candidates(g)[1:6]:
        print( "\t(" + x + ", " + str(WORD_IDX[x]) + ")" )

ProbabilityForWord("princetown")

# credits to: https://norvig.com/spell-correct.html

[i] ProbabilityForWord("the"): 0.026366561125148104
[i] ProbabilityForWord("pineapple"): 3.2780639608555458e-06
[i] pineapple occurs 9 times in all datasets
[i] all words: 2745523

[word: toothhick] best guess: toothpicks
probability: 2.185375973903697e-06
[word: toothhick] 5 other possible guesses toothhick
	(toothstick, 2)
	(toothpick, 7)
	(tothick, 1)
	(toothy, 7)
	(tooths, 4)

[word: princeron] best guess: princetown
probability: 1.0926879869518486e-06
[word: princeron] 5 other possible guesses princeron
	(princetons, 1)
	(princeton, 6)
	(Princeton, 5)
	(princedom, 4)
	(princekin, 3)

[word: jupytir] best guess: jupiter
probability: 2.5496053028876466e-06
[word: jupytir] 5 other possible guesses jupytir
	(jupitar, 1)
	(jupatis, 1)
	(juplter, 1)
	(jupati, 3)
	(upstir, 2)

[word: mashuo] best guess: smashup
probability: 1.0926879869518486e-06
[word: mashuo] 5 other possible guesses mashuo
	(mashlum, 2)
	(mashful, 1)
	(machuto, 1)
	(mashkov, 1)
	(masculo, 1)

[word: pineappee] best gu

1.0926879869518486e-06